In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
inputfile = "GaiaSource_1584380076484244352_2200921635402776448.csv"

In [3]:
colnames = ['solution_id', 'designation', 'source_id', 'random_index', 'ref_epoch', 
            'ra', 'ra_error', 'dec', 'dec_error', 
            'parallax', 'parallax_error', 'parallax_over_error', 
            'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 
            'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr', 
            'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 
            'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr',
            'astrometric_n_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_good_obs_al', 
            'astrometric_n_bad_obs_al', 'astrometric_gof_al', 'astrometric_chi2_al', 
            'astrometric_excess_noise', 'astrometric_excess_noise_sig', 
            'astrometric_params_solved', 'astrometric_primary_flag', 'astrometric_weight_al',
            'astrometric_pseudo_colour', 'astrometric_pseudo_colour_error', 
            'mean_varpi_factor_al', 'astrometric_matched_observations', 
            'visibility_periods_used', 'astrometric_sigma5d_max', 
            'frame_rotator_object_type', 'matched_observations', 'duplicated_source', 
            'phot_g_n_obs', 'phot_g_mean_flux', 'phot_g_mean_flux_error', 
            'phot_g_mean_flux_over_error', 'phot_g_mean_mag', 'phot_bp_n_obs', 
            'phot_bp_mean_flux', 'phot_bp_mean_flux_error', 'phot_bp_mean_flux_over_error', 
            'phot_bp_mean_mag', 'phot_rp_n_obs', 'phot_rp_mean_flux', 
            'phot_rp_mean_flux_error', 'phot_rp_mean_flux_over_error', 'phot_rp_mean_mag', 
            'phot_bp_rp_excess_factor', 'phot_proc_mode', 'bp_rp', 'bp_g', 'g_rp', 
            'radial_velocity', 'radial_velocity_error', 'rv_nb_transits', 
            'rv_template_teff', 'rv_template_logg', 'rv_template_fe_h', 'phot_variable_flag', 
            'l', 'b', 'ecl_lon', 'ecl_lat', 'priam_flags', 'teff_val', 
            'teff_percentile_lower', 'teff_percentile_upper', 'a_g_val', 
            'a_g_percentile_lower', 'a_g_percentile_upper', 'e_bp_min_rp_val', 
            'e_bp_min_rp_percentile_lower', 'e_bp_min_rp_percentile_upper', 'flame_flags', 
            'radius_val', 'radius_percentile_lower', 'radius_percentile_upper', 'lum_val', 
            'lum_percentile_lower', 'lum_percentile_upper']

In [4]:
# ra, dec in deg
# ra_err, dec_err in mas
# pm in mas/year, parallax in mas
# radial vel in km/s

def calculate_proper_motion(pmra, pmdec, dec):
    return np.sqrt(pmdec*pmdec + (pmra*math.cos(math.radians(dec)))**2)

def calculate_vt(pm, parallax):
    return 4.74*pm/parallax

def calculate_v(vt, vr):
    return np.sqrt(vt*vt + vr*vr)

def find_hvs(dec, pmra, pmdec, parallax, vr, limit=300):
    pm = calculate_proper_motion(pmra, pmdec, dec)
    vt = calculate_vt(pm, parallax)
    v = calculate_v(vt, vr)
    if v >= limit:
        return True, pm, vt, v
    else:
        return False, pm, vt, v

In [5]:
count = 0
noparallax = 0
hvsdata = []
with open(inputfile, "r") as ifile:
    header = ifile.readline()
    for line in ifile:
        dataline = line.strip().split(",")
        
        solution_id         = dataline[0]
        designation         = dataline[1]
        source_id           = dataline[2]
        random_index        = dataline[3]
        ref_epoch           = float(dataline[4])
        ra                  = float(dataline[5])
        ra_error            = float(dataline[6])
        dec                 = float(dataline[7])
        dec_error           = float(dataline[8])
        if dataline[9] != "":
            parallax            = float(dataline[9])
            parallax_error      = float(dataline[10])
            parallax_over_error = float(dataline[11])
            pmra                = float(dataline[12])
            pmra_error          = float(dataline[13])
            pmdec               = float(dataline[14])
            pmdec_error         = float(dataline[15])
            
            radial_velocity     = float(dataline[66])
            radial_velocity_error = float(dataline[67])
        
            # do something for this object/line
            # e.g. find HVS
            hvs, pm, vt, v = find_hvs(dec, pmra, pmdec, parallax, radial_velocity)
            if hvs:
                dataline.append([pm, vt, v])
                hvsdata.append(dataline)
        else:
            noparallax += 1

        count += 1
        
#         # for testing
#         if count > 30:
#             break

In [6]:
print count
print noparallax
print len(hvsdata)

1000000
4949
20364


In [7]:
for i in hvsdata:
    print i[-1]

[67.674593308680258, 315.4785558254128, 316.68034913872793]
[60.296205168455998, 254.53829076740234, 313.03596133850959]
[5.5478676976566357, 318.241120479188, 359.60433768204956]
[13.544754074283524, 355.12552244649902, 427.63313994433332]
[35.450658287528917, 335.14717712709898, 349.21659935342575]
[8.7221773995368608, 313.65260397775103, 313.67135696285942]
[5.7459945246073074, 337.21181088783555, 423.09852678785029]
[5.9690912935477929, 326.1659986618476, 338.26231342367845]
[60.729824429340319, 394.24033801793149, 396.54454562312492]
[56.569313713876916, 290.67773905004589, 304.73217253810037]
[34.034029897479193, 390.10360370759139, 427.02713031775005]
[1.9602859583300765, 643.63442040479242, 652.76131075539763]
[5.149436500865324, 461.06563015086857, 472.99352035614686]
[6.2311137173972933, 233.5703612670809, 301.20309028967461]
[3.4451412800412897, -1216.1815697391421, 1225.3046882639394]
[19.376576506349483, 476.90291278218359, 491.40365279963578]
[7.7002931847553091, 420.8795

[12.102836699585051, 374.92986758365009, 380.97863304531614]
[138.39670242496868, 276.95301403974315, 317.13126096475588]
[1.42184864727622, -407.83948235181958, 456.05187897937833]
[44.521769883322591, 369.1818176259319, 383.31045084824819]
[1.9879341861884656, 1725.1451347998614, 1727.6508348045545]
[1.7288567146161709, 388.69906521844075, 535.65712201054362]
[15.681541597809112, 263.160778487154, 334.31469803492757]
[17.925540771401277, -755.56572270181971, 755.86585031318145]
[11.219742808134095, 289.5446557009866, 482.18329390287528]
[34.837012003674069, 300.41888957057114, 346.83808489103313]
[15.021847622986677, 374.90345171000769, 476.98241994643985]
[2.9601362606021886, 341.77561812579256, 418.37049520679165]
[7.4361193247227968, 329.09865612057536, 341.91892398703175]
[2.4490332143476987, 507.67687071748463, 540.91958860456077]
[31.942659960982876, 370.76536812502871, 490.83221117464842]
[2.974433660836914, 149.5806713328665, 332.4329814625101]
[2.8808549777764441, 91.3782185

[9.188178581524399, 447.33446638074201, 474.16076101728703]
[12.510328739465148, 790.13555318683586, 818.752997855069]
[6.2099322792726817, 230.09830830388259, 329.52165444381643]
[5.1206347778557415, 17.931383127396504, 321.40221665286145]
[10.52008992719065, 329.46700045766596, 407.97484466792201]
[2.1358336870835335, 198.68915854667415, 368.88316503278571]
[4.4305791324417658, 428.12263624376897, 443.56419458743773]
[2.3585775005942549, 106.40827201624442, 343.80445724178759]
[16.032015313972764, 328.16229368008948, 344.54458322456657]
[11.488860123984306, 573.77373209982397, 580.37928077177298]
[20.334548785909448, -859.22276773946658, 859.22283808479062]
[7.4284336163573688, -446.22168199929217, 447.10286849840452]
[5.7466575783018472, 291.43693811038844, 330.83866436264708]
[35.016545400799409, 16.448599102110258, 402.53636777559632]
[4.0819810408109474, 262.15229110907569, 567.50474300892188]
[6.1640121728832611, 281.75976921261849, 587.29638084567841]
[5.0338286994525374, -560.

[13.728536940424824, 118.87184728495353, 384.0382571238647]
[5.8686355087815709, 612.73279845712284, 612.74283032825451]
[3.6980304144909311, 322.5747248797324, 327.02468511055605]
[5.1481591896719072, 542.50762155668485, 549.73894633513146]
[8.230812747753248, -378.77905041579555, 378.82007143686025]
[14.322597055378312, 298.89399936894102, 303.54742440687295]
[6.4956089354205195, 298.76143893984704, 314.24048443820232]
[6.9601824474187115, 407.75212524512062, 429.44951569974575]
[7.4094356430173098, 323.29063873785515, 324.79858527910909]
[4.6721829935138279, 560.1892970231861, 562.22248862850392]
[5.5452861295488987, 508.84954263295776, 511.52087328644961]
[7.3004297103387046, 348.52366117763313, 355.48359238239499]
[3.8546784693941962, -320.50651909170654, 334.90733241918355]
[6.4239930195932846, 391.28880772454272, 391.48724539318249]
[8.1658055877452398, 448.36508114285044, 475.77478939491436]
[7.151449278815325, 411.09980407691489, 414.10185113794819]
[9.8411233632563615, 346.35

[4.6686952357676628, 348.68158462854524, 348.68667549542261]
[4.2277150055991477, 471.96305248687548, 568.3398536499376]
[24.271626718919677, -1334.7713234533028, 1335.343461057693]
[9.7202034183988655, 555.50918094395968, 556.89479300959204]
[9.0499762273644038, 343.29728524318506, 343.46450616972817]
[5.4017921578403021, -24415.449365557713, 24415.464928118941]
[6.9248975419863772, -1490.5949495245738, 1490.5955696835324]
[6.0350626053137733, 10249.990820074585, 10250.02628347334]
[6.0274817102846407, 1013.8341948247696, 1013.85511479361]
[6.2351655915308708, 417.44794930412786, 417.46247917676351]
[5.8785011637834677, 401.08519511445417, 403.92365076594382]
[5.9715000354779795, 1045.6031117888124, 1047.0133816990597]
[8.0732056236771346, -2006.5356296458597, 2006.5385226099045]
[9.8062959046677793, -8432.7429279335902, 8432.8423706653775]
[4.9649776164199082, -368.17582641430073, 368.2232550471216]
[7.6058267758851441, 4617.9165902703944, 4618.0261626663469]
[6.5716794584366225, 406

...

[5.1217463810280828, 1921.4988865392277, 1921.8187863912547]
[3.446008274962395, 323.52206319659444, 336.06160707008468]
[5.2387470225357662, 314.55012975021123, 317.53178240477524]
[5.6396182325555371, 649.23350097944478, 735.56174923143396]
[3.9532045424241256, 695.33531301602545, 697.84869624759347]
[5.7745210413149781, 313.13017435966765, 313.19417373868401]
[4.2318125458787126, 1082.2529043577681, 1085.1010042795303]
[5.8690949054692014, -5223.9409568404953, 5224.7265162423537]
[6.7710664372070228, 467.89906054376331, 469.96732678187487]
[4.898514334606948, 1398.4997113333029, 1410.0646379694783]
[8.3336391029986068, 185.49359989338413, 342.52387661319591]
[4.8707685899627524, 307.92526240903203, 307.95816702300738]
[7.4033345500128398, 332.67666496165839, 352.77806726747383]
[10.935133265582506, 417.50759004473008, 428.8541749968619]
[9.9281483570187934, 361.50674689671104, 381.01505914118627]
[7.7312759960708144, 407.94497373745122, 408.2855314987853]
[6.6920533064436496, 372.09

[4.1238398966975156, 426.16501900827296, 435.254605326807]
[4.5435142753758138, 1761.5870794595014, 1763.1460474344578]
[5.2440733639561206, 729.03273649216976, 735.1854369065976]
[4.4023037304448618, 345.51032601053925, 349.45731923627108]
[9.100756242880923, -806.41546771855712, 806.77781969425212]
[5.2961209489869541, 386.34423032740762, 389.00831936320776]
[5.92842304691652, 505.44980371438965, 505.46433646113718]
[6.7317618852267547, 4999.3478266009279, 4999.4133596127022]
[4.7064002226290382, 346.24717374141113, 347.62163511837616]
[3.3051194886673327, -984.25107136111001, 993.90656641344481]
[6.120796295945329, -1259.0686912562317, 1259.4339830256972]
[5.6122693431452326, 398.51542409590559, 398.54843338369392]
[2.6657739140448991, 377.44152237847914, 383.74585520478485]
[4.4485948848803316, 312.13141928886648, 312.80185966890787]
[6.3947371100241801, 389.09247172079944, 396.03694951523676]
[8.4440629590384635, 342.18963626791276, 345.6586467293755]
[6.6343613136021586, 296.7314

[9.1335306719244986, 281.69557353262434, 304.82288975711538]
[8.8455354242451918, 395.39849993542794, 397.08064575871305]
[38.653326720346534, 412.14475132533187, 477.32828763291997]
[16.733447158537984, 36.150928298930666, 374.35482830771144]
[3.9240479171699829, 358.54876226999772, 361.01927355094023]
[4.1103578412406021, 1089.1205500038434, 1098.802246393667]
[8.224919651764754, 203.45798937470644, 300.15656270914405]
[4.2930784726222226, 501.8444843840997, 525.37346414496039]
[4.1259550789094401, 432.59116922590357, 440.93605551582033]
[4.9608255902107752, 319.41236626649254, 331.13553951306125]
[5.1956581741807719, 413.12751707039445, 422.60184836854421]
[5.0226102733131812, 330.05332381528382, 332.49353980725482]
[3.8062203912451102, 341.46413281450793, 346.61492539519145]
[20.85420060861237, 363.83080462452023, 467.43977752878391]
[6.7667028120561898, 304.44945249238668, 305.38715837631207]
[4.3355697420140746, 521.22085441511535, 529.04502165166559]
[5.3925974932106353, 364.537

[14.459111468599914, 274.54212207029161, 368.47165800253754]
[8.4046043434574642, 381.97807878766758, 415.06009968179939]
[8.6171523168609152, 917.43605409913243, 920.98066175483211]
[9.4879732052607419, -1042.6654967189791, 1042.8111043439276]
[8.9051775676858131, 457.2836153324713, 465.41727753179805]
[3.611170448046678, 434.66673526293698, 447.69397853003721]
[5.5113372212203133, 349.16022989818026, 374.91706901508337]
[6.3329823493228465, 314.81148323872429, 344.97981364234153]
[7.120816411412064, 297.77293387247511, 305.8872830258822]
[7.7881247246627865, 302.18858678881872, 304.22894330678776]
[18.234106525394196, 1244.6763486904547, 1245.5342396871167]
[12.48176882389131, 333.19305792050847, 336.52521470242061]
[6.1613560274465486, 385.36254313316323, 395.57005927987217]
[3.0462032911679309, -3478.4246997072223, 3479.3560054686268]
[8.4166567259454741, 449.31540209516561, 536.58414674709377]
[8.0638600023498128, 328.22752555879651, 330.72213891940731]
[6.5215590024879937, 318.36

[3.210031288411066, 352.58087119715702, 363.2481557799855]
[5.0627852568172074, 356.31638899625545, 358.49760859484951]
[4.7344404648224234, -4642.2473252030559, 4642.7338902671227]
[5.8520693249673696, 326.17236427164511, 337.87351615107821]
[4.6434430344347417, -529.45389890744968, 533.03412159335971]
[3.8641537546733842, -952.92775841364357, 956.1191816638725]
[3.6631492796154368, 429.76668540446087, 433.52890217286568]
[3.8471757593600486, 986.67834466300656, 988.05253856793797]
[3.6137237841233483, 372.07128317206292, 381.08539513093353]
[3.4355298425647649, -1427.7550700425161, 1429.5504550998196]
[5.6960934667167518, 1488.4967543998455, 1488.9057096003814]
[5.25880897133964, 2416.4338200137181, 2418.3503453409949]
[5.4169970364895246, 670.82219461790226, 670.88919291181355]
[3.7538555968649847, 3009.9036443575851, 3012.3621180046525]
[4.3653032786641219, 344.50323812465246, 356.77259337014777]
[3.8811680075715018, 1267.0183026302893, 1272.7143852683914]
[3.6088442374799214, 305.

In [12]:
hvsdata[-1]

['1635721458409799680',
 'Gaia DR2 2200723963825920640',
 '2200723963825920640',
 '210585923',
 '2015.5',
 '341.0333416042042',
 '0.052646554570414936',
 '60.0574816980786',
 '0.045574764431980615',
 '0.003816699960380127',
 '0.0570050993219381',
 '0.06695366',
 '-3.918833557771176',
 '0.0950206094350409',
 '-2.754476085845754',
 '0.08731795710474083',
 '0.12158433',
 '0.2635712',
 '-0.4521276',
 '-0.11444073',
 '0.011551698',
 '-0.14807199',
 '-0.23447962',
 '-0.22132054',
 '0.13506836',
 '0.10953441',
 '245',
 '0',
 '245',
 '0',
 '20.71456',
 '1038.2281',
 '0.44947366527647264',
 '28.563580115451',
 '31',
 'false',
 '3.987427',
 '1.2371490756824204',
 '0.01368639208453343',
 '0.08194647',
 '28',
 '18',
 '0.09787696',
 '0',
 '31',
 'false',
 '266',
 '19243.633610015404',
 '13.281803905267854',
 '1448.872',
 '14.977648',
 '28',
 '1864.5197957425003',
 '18.284302449715195',
 '101.9738',
 '17.17497',
 '28',
 '29903.13823070211',
 '122.37731996029724',
 '244.35196',
 '13.572628',
 '1.6508